Лабораторна робота №4

Роботу виконали студенти групи ФБ - 35: Церман Марія, Кохта Андрій, Ворона Сергій.

In [1]:
import random

Генерація простих чисел та розбиття на пари

In [2]:
def power(a, b, m):
    res = 1
    a %= m
    while b > 0:
        if b % 2 == 1:
            res = (res * a) % m
        a = (a * a) % m
        b //= 2

    return res

def miller_rabin_test(n, k=40):
    if n == 2 or n == 3:
        return True
    if n % 2 == 0 or n < 2:
        return False

    s = 0
    d = n - 1
    while d % 2 == 0:
        d //= 2
        s += 1

    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = power(a, d, n)

        if x == 1 or x == n - 1:
            continue

        for r in range(s - 1):
            x = power(x, 2, n)
            if x == n - 1:
                break
        else:
            return False

    return True


def get_random_prime(bits):
    while True:
        candidate = random.getrandbits(bits)
        candidate |= (1 << (bits - 1)) | 1

        small_primes = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53]
        is_composite = False
        for p in small_primes:
            if candidate % p == 0 and candidate != p:
                is_composite = True
                break

        if is_composite:
            continue

        if miller_rabin_test(candidate):
            return candidate

In [3]:
p, q, p1, q1 = get_random_prime(256), get_random_prime(256), get_random_prime(256), get_random_prime(256)

In [4]:
def check_keys(p, q, p1, q1):
    n = p * q
    n1 = p1 * q1

    if n > n1:
        return (p1, q1), (p, q), n1, n
    else:
        return (p, q), (p1, q1), n, n1

In [5]:
pair_A, pair_B, mult_A, mult_B = check_keys(p, q, p1, q1)
print("Пара для абонента А:")
print(f"p: {pair_A[0]}\n"
      f"q: {pair_A[1]}\n"
      f"pq: {mult_A}\n")

print("Пара для абонента B:")
print(f"p1: {pair_B[0]}\n"
      f"q1: {pair_B[1]}\n"
      f"pq1: {mult_B}\n")

print("pq < pq1:", ["Ні", "Так"][mult_A <= mult_B])

Пара для абонента А:
p: 61285799349330010685056755947923214890714928272535135781031639415192272233991
q: 58749625129276473123355180996424906866902340238389124125151936647145666160763
pq: 3600517737521194121420077081320642873770122293129387574375459125291623005043271658289783588888368028366210005448349225729574686189461636799968794159095133

Пара для абонента B:
p1: 68171730501724312868827841655001528403549101379577761743814381837115158062001
q1: 110069901398657882690490470528954666564471068640304332883071209951866604553663
pq1: 7503655654500673187929048704110657142928172296035882835160673513565931123161493678877236452376156748133602321668188359647764536388908781691803072685659663

pq < pq1: Так


Генерація пар ключів(публічного та секретного) для RSA

In [6]:
def euclid_extended(a, n):
    r0, r1 = a, n
    u0, v0 = 1, 0
    u1, v1 = 0, 1

    while r1 != 0:
        q = r0 // r1
        (r0, r1) = (r1, r0 - q * r1)

        (u0, u1) = (u1, u0 - q * u1)
        (v0, v1) = (v1, v0 - q * v1)

    gcd = r0
    u = u0
    v = v0

    return gcd, u, v

def generate_key_pair(p, q):
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    gcd, u, _ = euclid_extended(e, phi)

    while gcd != 1:
        e = random.randrange(3, phi, 2)
        gcd, u, _ = euclid_extended(e, phi)

    d = u % phi

    return (e, n), (d, p, q)

In [7]:
public_A, private_A = generate_key_pair(pair_A[0], pair_A[1])
public_B, private_B = generate_key_pair(pair_B[0], pair_B[1])

for pub, priv, name in [(public_A, private_A, "A"), (public_B, private_B, "B")]:
    e, n = pub
    d, p, q = priv

    print(f"Абонент {name}:")
    print("Відкритий ключ")
    print(f"e: {e}\n"
          f"n: {n}\n")

    print("Приватний ключ")
    print(f"d: {d}\n"
          f"p: {p}\n"
          f"q: {q}\n")

Абонент A:
Відкритий ключ
e: 65537
n: 3600517737521194121420077081320642873770122293129387574375459125291623005043271658289783588888368028366210005448349225729574686189461636799968794159095133

Приватний ключ
d: 2677327782819176238142798974686038255755365209134907680577982659457049665147442025681598298785745514555860098547336448954529812321482916550355270015462893
p: 61285799349330010685056755947923214890714928272535135781031639415192272233991
q: 58749625129276473123355180996424906866902340238389124125151936647145666160763

Абонент B:
Відкритий ключ
e: 65537
n: 7503655654500673187929048704110657142928172296035882835160673513565931123161493678877236452376156748133602321668188359647764536388908781691803072685659663

Приватний ключ
d: 608884153541275615505846788965934886950760948324134838600858473002176201427771724297277831167438502400022931589584292313390086081709433197428550826689473
p: 68171730501724312868827841655001528403549101379577761743814381837115158062001
q: 11006990139865788269

Шифрування, розшифрування, цифровий підпис, перевірка

In [8]:
def Encrypt(message, public_key):
    e, n = public_key
    if message >= n:
        raise ValueError("Помилка: Повідомлення M має бути меншим за модуль n")

    return power(message, e, n)

def Decrypt(ciphertext, private_key):
    if len(private_key) == 3:
        d, p, q = private_key
        n = p * q
    else:
        d, n = private_key

    return power(ciphertext, d, n)

def Sign(message, private_key):
    if len(private_key) == 3:
        d, p, q = private_key
        n = p * q
    else:
        d, n = private_key

    return power(message, d, n)

def Verify(message, signature, public_key):
    e, n = public_key
    check_val = power(signature, e, n)

    return check_val == message

In [9]:
subscribers = [("А", public_A, private_A), ("В", public_B, private_B)]

for name, pub_key, priv_key in subscribers:
    # Генерація повідомлення
    n_module = pub_key[1]
    message = random.randint(100, n_module // 2)
    print(f"Відкрите повідомлення {name}: {message}")

    # Шифрування
    crypto = Encrypt(message, pub_key)
    print(f"Криптограма {name}: {crypto}")

    # Розшифрування
    decrypted = Decrypt(crypto, priv_key)
    print(f"Розшифроване повідомлення {name}: {decrypted}")
    print(["Не співпадають\n", "Співпадають\n"][decrypted == message])

    # Цифровий підпис
    signature = Sign(message, priv_key)
    print(f"Цифровий підпис повідомлення {name}: {signature}")

    # Перевірка підпису
    is_valid = Verify(message, signature, pub_key)
    status = (["!Невалідний!", "!Валідний!"][is_valid])
    print(f"Підпис {name}: {status}\n")

Відкрите повідомлення А: 107725376100711466714603655766153621741661956299991237155622452161713093112006981361212765912653889502125305876965261603545695164339505108382985555847159
Криптограма А: 185749075340260875984269686974440870973958813633193998630578255251613967294507520328358567948608881374803686321780704232163579461859431881836870160582028
Розшифроване повідомлення А: 107725376100711466714603655766153621741661956299991237155622452161713093112006981361212765912653889502125305876965261603545695164339505108382985555847159
Співпадають

Цифровий підпис повідомлення А: 1678699544938803616055390463789242628522892892595912894690217749581750392449987882313222277129810783667957681748595166045388389212965273513713400390071161
Підпис А: !Валідний!

Відкрите повідомлення В: 3317383768000244629044417069456234602370328920888191946750287660440053871691262730290903996740788479497004540112564271422929651065831042317964865930131502
Криптограма В: 3690534436632652161101787763432116392029119970291568

Конфіденційне розсилання ключів

In [10]:
def SendKey(k, private_key_A, public_key_B):
    S = Sign(k, private_key_A)
    k1 = Encrypt(k, public_key_B)
    S1 = Encrypt(S, public_key_B)

    return k1, S1

def ReceiveKey(package, private_key_B, public_key_A):
    k1, S1 = package
    k = Decrypt(k1, private_key_B)
    S = Decrypt(S1, private_key_B)
    is_valid = Verify(k, S, public_key_A)

    if is_valid:
        return k, "Підпис вірний, ключ отримано!"
    else:
        return None, "Підпис невірний!"

In [11]:
# 0<k<n
n_a = public_A[1]
secret_k = random.randint(100, n_a // 2)
print(f"Секретний ключ k: {secret_k}\n")

print("Відправляє А:")
package = SendKey(secret_k, private_A, public_B)
print(f"Шифрований ключ: {package[0]}\n"
      f"Цифровий підпис: {package[1]}\n")

print("Отримує B:")
received_k, status_msg = ReceiveKey(package, private_B, public_A)
print(f"Відкритий ключ k: {received_k}")
print(status_msg)

Секретний ключ k: 1223529766078021391120989571659784350363344031566575137701459055187667541404274264023104405153579587139382781155646746658856881893473183649867198117829189

Відправляє А:
Шифрований ключ: 2247568379434742932364874637519177500555029095683329470245767756444485719932166766698883775651855311347357797554954496565018485217532123860227494571388674
Цифровий підпис: 4379937692547364285506198314749280817822249740833173954876001094506792544988815045130370325992622513731483607336372438789731589086462826772884655330291197

Отримує B:
Відкритий ключ k: 1223529766078021391120989571659784350363344031566575137701459055187667541404274264023104405153579587139382781155646746658856881893473183649867198117829189
Підпис вірний, ключ отримано!
